In [10]:
date='0704'

WIN PROBABILITY DATA

In [ ]:
import pandas as pd
import requests
from audl.stats.endpoints.gameevents import *
import numpy as np

features = ['thrower_x', 'thrower_y', 'start_on_offense', 'point_start_time', 'possession_num', 'possession_throw', 'game_quarter', 'quarter_point', 'is_home_team', 'home_team_score', 'away_team_score', 'gameID', 'total_points', 'home_teamID', 'away_teamID']

def process_games(GAMES):
    GAMES['total_points'] = GAMES['home_team_score'] + GAMES['away_team_score']
    GAMES = GAMES[GAMES.game_quarter < 5] ##TODO include overtimes
    GAMES['point_start_time'] = 720 - GAMES['point_start_time']
    GAMES = GAMES[features]

    games = []
    for gameID in GAMES.gameID.unique():
        GAME = GAMES[GAMES.gameID == gameID]
        points = []
        prev_point_time = 720
        current_quarter = 1
        prev_df = None
        error_points = 0
        for group_keys, group_df in GAME.groupby(['total_points', 'game_quarter']):
            if prev_df is None:
                prev_df = group_df
                continue

            prev_point_time = prev_df.point_start_time.max()
            if (current_quarter != group_df.game_quarter.max()):
                current_quarter = group_df.game_quarter.max()
                current_point_time = 0
            else:
                current_point_time = group_df.point_start_time.max()
            if current_point_time >= prev_point_time:
                error_points = error_points + 1
                continue
            times = np.linspace(prev_point_time, current_point_time + 1, len(prev_df))
            prev_df['times'] = times
            points.append(prev_df)
            prev_df = group_df
            if group_keys == (GAME.home_team_score.max()+GAME.away_team_score.max(), 4):
                times = np.linspace(current_point_time, 0, len(prev_df))
                prev_df['times'] = times
                points.append(prev_df)
        GAME_POINTS = pd.concat(points)
        GAME_POINTS.times = GAME_POINTS.times + ((4 - GAME_POINTS.game_quarter)*720)
        try:
            # if error_points > 0:
            #     print(f'{error_points} errors with {GAME_POINTS.gameID.iloc[0]}')
            assert GAME_POINTS.times.is_monotonic_decreasing, f'timing is off somewhere for game: {GAME_POINTS.gameID.iloc[0]}'
            home_team_win = GAME_POINTS[['away_team_score', 'home_team_score']].max().values.argmax() if GAME_POINTS['home_team_score'].max() != GAME_POINTS['away_team_score'].max() else -2
            GAME_POINTS['home_team_win'] = home_team_win
            games.append(GAME_POINTS)
        except AssertionError as e:
            print(e)

        PROCESSED_GAMES = pd.concat(games).drop(['point_start_time', 'start_on_offense'], axis=1)
        PROCESSED_GAMES['score_diff'] = PROCESSED_GAMES['home_team_score'] - PROCESSED_GAMES['away_team_score']
    return PROCESSED_GAMES[PROCESSED_GAMES.home_team_win >= 0]

base_url = "https://www.backend.audlstats.com/api/v1/"

endpoint = "games?date=2021:"
ids = pd.DataFrame(requests.get(f'{base_url}{endpoint}').json()['data'])['gameID'].values

game_events_proxy = GameEventsProxy()
all_games = []
for id in ids:
    try:
        game_events = GameEvents(id, game_events_proxy)
        game_events.process_game_events()
        game_df = game_events.get_events_df(True, True, True)
        all_games.append(game_df)
    except AssertionError as e:
        print(id)
        print(e)
    except Exception as e:
        print(id)
        raise e
ALL_GAMES = pd.concat(all_games)
ALL_GAMES.to_csv(f'../data/raw/all_games_{date}.csv', index=False)
df = process_games(ALL_GAMES)
df.to_csv(f'../data/processed/all_games_{date}.csv', index=False)


PLAYER DATA

In [12]:
from audl.stats.endpoints.playerstats import PlayerStats
import pandas as pd
year = 2021
dfs = []
while True:
    df = PlayerStats(year, 'total', 'all').fetch_table()
    df['year'] = year
    dfs.append(df)
    year = year+1
    if len(df) == 0:
        break
player_stats_by_year = pd.concat(dfs)
player_stats_by_year.to_csv(f'../data/raw/player_stat_by_year_{date}')

In [85]:
from audl.stats.endpoints.players import Players
from audl.stats.endpoints.teams import Teams

teams = Teams()
teams_df = teams.get_teams()
teams_df = pd.merge(teams_df.drop('division', axis=1).reset_index(drop=True), teams_df.division.apply(pd.Series).reset_index(drop=True), left_index=True, right_index=True)
teams_df.to_csv(f'../data/raw/teams_{date}')

players = Players()
players_df = players.get_players()
players_df = players_df.explode('teams')
players_df = pd.merge(players_df.drop('teams', axis=1).reset_index(drop=True), players_df.teams.apply(pd.Series).reset_index(drop=True), left_index=True, right_index=True)
players_df['firstName'] = players_df['firstName'].apply(lambda x:str(x).strip())
players_df['lastName'] = players_df['lastName'].apply(lambda x:str(x).strip())
players_df.to_csv(f'../data/raw/players_{date}')

In [11]:
from audl.stats.endpoints.playergamestats import PlayerGameStats
import pandas as pd
playergamestats = PlayerGameStats()

In [19]:
gameID = '2023-06-24-DC-BOS'
game_stats_df = playergamestats.get_request_as_df(f'playerGameStats?gameID={gameID}')
game_stats_df = pd.merge(game_stats_df.player.apply(pd.Series), game_stats_df.drop('player', axis=1), left_index=True, right_index=True)
game_stats_df['fullName'] = game_stats_df['firstName'] + ' ' + game_stats_df['lastName']
stats_cols = ['playerID', 'full_name', 'assists', 'goals', 'hockeyAssists', 'completions', 'throwaways', 'stalls', 'yardsReceived', 'yardsThrown', 'hucksCompleted', 'drops',
 'blocks', 'callahans', 'oPointsPlayed', 'dPointsPlayed']

In [ ]:
['playerID', 'full_name', 'assists', 'goals', 'hockeyAssists', 'completions', 'throwaways', 'stalls', 'yardsReceived', 'yardsThrown', 'hucksCompleted', 'drops',
 'blocks', 'callahans', 'oPointsPlayed', 'dPointsPlayed']

In [1]:
from audl.stats.endpoints.gameevents import GameEventsProxy
game_events = GameEventsProxy()
game_events.get_throws_from_id('2023-06-24-DC-BOS')

,type,thrower,throwerX,throwerY,receiver,receiverX,receiverY,turnover,is_home_team
0,18,csunde,0.00,40.00,pboerth,-8.29,59.61,0,True
1,18,pboerth,-8.29,59.61,cdavisbra,-13.71,79.16,0,True
2,18,cdavisbra,-13.71,79.16,thalkyard,-21.64,97.22,0,True
3,18,thalkyard,-21.64,97.22,cdavisbra,-22.48,95.41,0,True
4,18,cdavisbra,-22.48,95.41,tchan,2.74,95.54,0,True
...,...,...,...,...,...,...,...,...,...
626,22,mdia,21.05,38.88,NaN,26.66,84.24,1,False
627,18,aroy,-26.67,40.41,jnissen,-11.44,33.49,0,False
628,18,jnissen,-11.44,33.49,tmonroe,5.75,32.83,0,False
629,22,tmonroe,5.75,32.83,NaN,1.89,37.64,1,False


In [14]:
rows = []
for event in game_events.current_request['homeEvents']:
    if event['type'] in [18, 19, 20, 22, 23]:
        turnover = 1 if event['type'] in [20, 22, 23] else 0
        event['turnover'] = turnover
        if 'receiverX' not in event:
            event['receiverX'], event['receiverY'] = event['turnoverX'], event['turnoverY']
        event['is_home_team'], event['turnoverX'], event['turnoverY'] = True, None, None
        rows.append(event)

for event in game_events.current_request['awayEvents']:
    if event['type'] in [18, 19, 20, 22, 23]:
        turnover = 1 if event['type'] in [20, 22, 23] else 0
        event['turnover'] = turnover
        if 'receiverX' not in event:
            event['receiverX'], event['receiverY'] = event['turnoverX'], event['turnoverY']
        event['is_home_team'], event['turnoverX'], event['turnoverY'] = False, None, None
        rows.append(event)

pd.DataFrame(rows).drop(['turnoverX', 'turnoverY'], axis=1

In [16]:
import pandas as pd
pd.DataFrame(rows).drop(['turnoverX', 'turnoverY'], axis=1)

KeyError: "['turnoverX', 'turnoverY'] not found in axis"

In [5]:
from audl.stats.endpoints.games import Games
games = Games()
games_df = games.get_games()

In [6]:
games_df

,gameID,awayTeamID,homeTeamID,awayScore,homeScore,status,startTimestamp,startTimezone,streamingURL,updateTimestamp,week
0,2023-06-24-DC-BOS,breeze,glory,20,16,Final,2023-06-24T17:00:00-04:00,EDT,https://audltv.vhx.tv/videos/dc-at-boston-6-24...,2023-06-30T21:25:44.248Z,week-9
1,2023-05-19-LA-SLC,aviators,shred,13,25,Final,2023-05-19T19:00:00-06:00,MDT,https://audltv.vhx.tv/videos/los-angeles-at-sa...,2023-05-29T05:43:47.960Z,week-4
2,2023-05-06-DAL-ATX,legion,sol,17,31,Final,2023-05-06T19:00:00-05:00,CDT,https://audltv.vhx.tv/videos/dallas-at-austin-...,2023-05-10T05:30:26.666Z,week-2
3,2023-06-10-SD-OAK,growlers,spiders,17,21,Final,2023-06-10T17:00:00-07:00,PDT,https://audltv.vhx.tv/videos/san-diego-at-oakl...,2023-06-13T20:02:58.682Z,week-7
4,2021-07-11-MIN-CHI,windchill,union,20,18,Final,2021-07-11T14:00:00-05:00,CDT,None,2021-07-11T21:06:19.323Z,week-6
...,...,...,...,...,...,...,...,...,...,...,...
434,2022-06-04-BOS-PHI,glory,phoenix,22,24,Final,2022-06-04T18:00:00-04:00,EDT,https://audltv.vhx.tv/videos/boston-at-philade...,2022-08-13T22:58:17.565Z,week-6
435,2022-07-22-LA-SLC,aviators,shred,16,19,Final,2022-07-22T19:00:00-06:00,MDT,https://audltv.vhx.tv/videos/los-angeles-at-sa...,2022-07-25T02:58:10.083Z,week-13
436,2022-07-30-POR-COL,nitro,summit,18,34,Final,2022-07-30T19:00:00-06:00,MDT,https://audltv.vhx.tv/videos/portland-at-color...,2022-07-31T03:18:38.274Z,week-14
437,2022-07-30-OTT-PHI,outlaws,phoenix,19,25,Final,2022-07-30T18:00:00-04:00,EDT,https://audltv.vhx.tv/videos/ottawa-at-philade...,2022-07-31T18:04:06.565Z,week-14
